### Hyperparameter tuning of Deep learning models in Keras

In [3]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
from keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras import optimizers

In [2]:
# Install and import aisaratuners
!pip install aisaratuners
from aisaratuners.aisara_keras_tuner import Hp, HpOptimization

In [4]:
# load fashion mnist dataset
(img_train, y_train), (img_test, y_test) = keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [5]:
y_test

array([9, 2, 1, ..., 8, 1, 5], dtype=uint8)

In [6]:
# Normalize pixel values between 0 and 1
x_train = img_train.astype('float32') / 255.0
x_test = img_test.astype('float32') / 255.0

In [7]:
y_train=to_categorical(y_train,)
y_test=to_categorical(y_test)

In [8]:
# define Hps:
hps = Hp()
hp_1 = hps.numrange(name='nodes_dense',min=64,max=512)
hp_2 = hps.numrange(name='lr',min=0.0001,max=0.01, type='log')

In [9]:
# define keras model and return model history
def model_builder(comp, k):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))
    #model.add(Dense(comp['nodes_dense'][k], activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(comp['nodes_dense'][k], activation='relu'))
    model.add(Dense(10, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(comp['lr'][k]),metrics=['acc'])
    history = model.fit(x_train,y_train,epochs=5,validation_data=(x_test,y_test))
    return model, history

In [10]:
# run optimizer
demo = HpOptimization(hps, model_builder, ['val_acc','val_loss'], ['max','min'], num_trials=3, rounds=1, mode='p')
%time demo.run_opti()

For commercial use, you can obtain our API from https://rapidapi.com/aisara-technology-aisara-technology-default/api/aisara-hyperparameter-tuning
If you are a private user, set the mode parameter in HpOptimization class to "p".

Round-1:

  Run-1:
Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.5197 - acc: 0.8126 - val_loss: 0.4702 - val_acc: 0.8301
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.4257 - acc: 0.8442 - val_loss: 0.4024 - val_acc: 0.8542
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.4003 - acc: 0.8530 - val_loss: 0.4192 - val_acc: 0.8457
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3863 - acc: 0.8567 - val_loss: 0.4188 - val_acc: 0.8466
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3735 - acc: 0.8633 - val_loss: 0.3769 - val_acc: 0.8639

  Run-2:
Epoch 1/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.

In [11]:
demo.best_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 784)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 139)               109115    
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1400      
Total params: 110,515
Trainable params: 110,515
Non-trainable params: 0
_________________________________________________________________


In [12]:
# optimization results in dataframe
print(demo.opti_results)

 nodes_dense        lr      loss       acc  val_loss  val_acc    Round   model ID
         288  0.004642  0.373524  0.863283  0.376903   0.8639  Round_1  model_1_1
         139  0.001000  0.338249  0.874383  0.340270   0.8746  Round_1  model_1_2
         437  0.000215  0.362038  0.868417  0.370761   0.8664  Round_1  model_1_3


In [13]:
from matplotlib import pyplot as plt
# visualize optimization results
demo.plot_opti_results()

In [14]:
# visualize hyperparameters search space
demo.plot_search_space()

In [15]:
# best model hyperparameters
print(demo.best_model_hps)

 nodes_dense     lr
         139  0.001


In [16]:
# loading the best model
best = demo.best_model
best.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 784)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 139)               109115    
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1400      
Total params: 110,515
Trainable params: 110,515
Non-trainable params: 0
_________________________________________________________________
